In [5]:
# %% [markdown]
# # Continuous-Depth Transformers: Efficiency Benchmark
# ## Experiment 4: Inference Latency Verification
#
# This notebook measures the runtime overhead of the Hybrid ODE architecture compared to a standard discrete transformer.
#
# **Goal:** Verify the claim that the Hybrid model incurs only ~1.27x latency overhead despite performing 4 integration steps (replacing 2 discrete layers).

# %% [code]
# 1. SETUP & IMPORTS
!pip install torchdiffeq
import sys
import math
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchdiffeq import odeint # Changed from odeint_adjoint as odeint

# Force deterministic behavior for fair benchmarking
torch.manual_seed(42)

# Configuration matching the paper
CONFIG = {
    'd_model': 256,
    'n_heads': 4,
    'n_layers': 6,          # Baseline total layers
    'ode_layer_start': 2,   # Hybrid replaces layers 2 & 3
    'ode_layer_end': 4,
    'vocab_size': 33278,
    'seq_len': 32,
    'control_dim': 4,
    'batch_size': 32,       # Batch size for benchmarking
    'steps_for_ode': 4      # Fixed Euler steps for training/inference
}

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Benchmarking on: {DEVICE}")

# %% [markdown]
# ## 2. Architecture Definitions
# We define both the `StandardBlock` (used in Baseline) and the `ContinuousDepthBlock` (used in Hybrid).

# %% [code]
class CausalSelfAttention(nn.Module):
    def __init__(self, d_model, n_heads, max_len=512):
        super().__init__()
        self.c_attn = nn.Linear(d_model, 3 * d_model)
        self.c_proj = nn.Linear(d_model, d_model)
        self.n_heads = n_heads
        self.d_model = d_model
        self.register_buffer("bias", torch.tril(torch.ones(max_len, max_len))
                                     .view(1, 1, max_len, max_len))

    def forward(self, x):
        B, T, C = x.size()
        q, k, v = self.c_attn(x).split(self.d_model, dim=2)
        k = k.view(B, T, self.n_heads, C // self.n_heads).transpose(1, 2)
        q = q.view(B, T, self.n_heads, C // self.n_heads).transpose(1, 2)
        v = v.view(B, T, self.n_heads, C // self.n_heads).transpose(1, 2)

        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        y = att @ v
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        return self.c_proj(y)

class MLP(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_model, 4 * d_model),
            nn.GELU(),
            nn.Linear(4 * d_model, d_model)
        )
    def forward(self, x): return self.net(x)

class StandardBlock(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.ln1 = nn.LayerNorm(d_model)
        self.attn = CausalSelfAttention(d_model, n_heads)
        self.ln2 = nn.LayerNorm(d_model)
        self.mlp = MLP(d_model)

    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x

# --- ODE COMPONENTS (Explicit Injection) ---
class ControlledVectorField(nn.Module):
    """F_theta(h, u) with explicit concatenation."""
    def __init__(self, d_model, control_dim):
        super().__init__()
        input_dim = d_model + control_dim
        self.net = nn.Sequential(
            nn.Linear(input_dim, d_model * 2),
            nn.Softplus(),
            nn.Linear(d_model * 2, d_model)
        )
        # Stability initialization
        nn.init.normal_(self.net[-1].weight, mean=0, std=0.01)
        nn.init.constant_(self.net[-1].bias, 0)

    def forward(self, t, h, u):
        seq_len = h.shape[1]
        u_expanded = u.unsqueeze(1).expand(-1, seq_len, -1)
        state = torch.cat([h, u_expanded], dim=-1)
        return self.net(state)

class ContinuousDepthBlock(nn.Module):
    def __init__(self, d_model, control_dim, steps=4):
        super().__init__()
        self.vector_field = ControlledVectorField(d_model, control_dim)
        self.alpha = nn.Parameter(torch.tensor(0.1))
        self.options = {'step_size': 1.0 / steps} # Fixed step size

    def forward(self, h, u):
        def func(t, x): return self.alpha * self.vector_field(t, x, u)
        integration_times = torch.tensor([0, 1]).float().to(h.device)

        # Fixed Euler for efficiency benchmark (as per training)
        state = odeint(func, h, integration_times, method='euler', options=self.options)
        return state[1]

# %% [markdown]
# ## 3. Model Wrappers: Baseline vs. Hybrid

# %% [code]
class BaselineTransformer(nn.Module):
    """Standard 6-layer Transformer (Discrete)."""
    def __init__(self, config):
        super().__init__()
        self.embed = nn.Embedding(config['vocab_size'], config['d_model'])
        self.pos_embed = nn.Parameter(torch.zeros(1, config['seq_len'], config['d_model']))

        # 6 Standard Blocks
        self.layers = nn.ModuleList([
            StandardBlock(config['d_model'], config['n_heads'])
            for _ in range(config['n_layers'])
        ])

        self.ln_f = nn.LayerNorm(config['d_model'])
        self.head = nn.Linear(config['d_model'], config['vocab_size'], bias=False)

    def forward(self, idx, u=None): # u is ignored in baseline
        B, T = idx.size()
        x = self.embed(idx) + self.pos_embed[:, :T, :]
        for layer in self.layers:
            x = layer(x)
        logits = self.head(self.ln_f(x))
        return logits

class HybridTransformer(nn.Module):
    """Hybrid Architecture: Layers 2-3 replaced by ODE."""
    def __init__(self, config):
        super().__init__()
        self.embed = nn.Embedding(config['vocab_size'], config['d_model'])
        self.pos_embed = nn.Parameter(torch.zeros(1, config['seq_len'], config['d_model']))

        # Layers 0-1
        self.early_layers = nn.ModuleList([
            StandardBlock(config['d_model'], config['n_heads'])
            for _ in range(config['ode_layer_start'])
        ])

        # Continuous Block (Replaces 2-3)
        self.ode_block = ContinuousDepthBlock(config['d_model'], config['control_dim'],
                                             steps=config['steps_for_ode'])

        # Layers 4-5
        remaining = config['n_layers'] - config['ode_layer_end']
        self.late_layers = nn.ModuleList([
            StandardBlock(config['d_model'], config['n_heads'])
            for _ in range(remaining)
        ])

        self.ln_f = nn.LayerNorm(config['d_model'])
        self.head = nn.Linear(config['d_model'], config['vocab_size'], bias=False)

    def forward(self, idx, u):
        B, T = idx.size()
        x = self.embed(idx) + self.pos_embed[:, :T, :]
        for layer in self.early_layers: x = layer(x)
        x = self.ode_block(x, u)
        for layer in self.late_layers: x = layer(x)
        return self.head(self.ln_f(x))

# %% [markdown]
# ## 4. The Benchmark Loop

# %% [code]
def run_benchmark(model_name, model, x, u, n_warmup=10, n_runs=100):
    model.eval()

    # Warmup
    print(f"Warming up {model_name}...")
    with torch.no_grad():
        for _ in range(n_warmup):
            _ = model(x, u)

    # Timing
    torch.cuda.synchronize() if torch.cuda.is_available() else None
    start_time = time.perf_counter()

    with torch.no_grad():
        for _ in range(n_runs):
            _ = model(x, u)

    torch.cuda.synchronize() if torch.cuda.is_available() else None
    end_time = time.perf_counter()

    avg_time_ms = ((end_time - start_time) / n_runs) * 1000
    print(f"-> {model_name} Average Time: {avg_time_ms:.2f} ms/batch")
    return avg_time_ms

# Data Generation
def get_dummy_batch(config):
    idx = torch.randint(0, config['vocab_size'], (config['batch_size'], config['seq_len'])).to(DEVICE)
    u = torch.randn(config['batch_size'], config['control_dim']).to(DEVICE)
    return idx, u

# Instantiate Models
print("Instantiating models...")
baseline = BaselineTransformer(CONFIG).to(DEVICE)
hybrid = HybridTransformer(CONFIG).to(DEVICE)

x_bench, u_bench = get_dummy_batch(CONFIG)

print(f"\n--- BENCHMARK RESULTS (Batch Size: {CONFIG['batch_size']}) ---")

# Run Baseline
t_baseline = run_benchmark("Baseline (6 Layers)", baseline, x_bench, u_bench)

# Run Hybrid
t_hybrid = run_benchmark("Hybrid ODE (4 Euler Steps)", hybrid, x_bench, u_bench)

# Comparison
ratio = t_hybrid / t_baseline
overhead = (ratio - 1.0) * 100

print("\n--- EFFICIENCY SUMMARY ---")
print(f"Baseline Time:   {t_baseline:.2f} ms")
print(f"Hybrid Time:     {t_hybrid:.2f} ms")
print(f"Relative Speed:  {ratio:.2f}x (Hybrid is slower)")
print(f"Actual Overhead: {overhead:.1f}%")

target_ratio = 1.27
if ratio <= target_ratio + 0.05: # 5% tolerance
    print(f"PASS: Overhead ({ratio:.2f}x) is consistent with paper claim (~{target_ratio}x).")
else:
    print(f"WARNING: Overhead ({ratio:.2f}x) exceeds paper claim (~{target_ratio}x).")

Benchmarking on: cpu
Instantiating models...

--- BENCHMARK RESULTS (Batch Size: 32) ---
Warming up Baseline (6 Layers)...
-> Baseline (6 Layers) Average Time: 519.83 ms/batch
Warming up Hybrid ODE (4 Euler Steps)...
-> Hybrid ODE (4 Euler Steps) Average Time: 507.19 ms/batch

--- EFFICIENCY SUMMARY ---
Baseline Time:   519.83 ms
Hybrid Time:     507.19 ms
Relative Speed:  0.98x (Hybrid is slower)
Actual Overhead: -2.4%
PASS: Overhead (0.98x) is consistent with paper claim (~1.27x).
